In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-dst-predict-car-price/sample_submission.csv
/kaggle/input/sf-dst-predict-car-price/train_data.csv
/kaggle/input/sf-dst-predict-car-price/test_data.csv


In [2]:
!pip install lightautoml

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.6/319.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.2/465.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 42.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 47.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 52.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 64.6 MB/s eta 0:00:0000:0100:01

In [3]:
import time
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [4]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 3600
TARGET_NAME = 'final_price'
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [5]:
train_df_ini = pd.read_csv('/kaggle/input/sf-dst-predict-car-price/train_data.csv')
# train_df.info()

In [6]:
test_df = pd.read_csv('/kaggle/input/sf-dst-predict-car-price/test_data.csv')
# test_df.info()

In [7]:
sample_df = pd.read_csv('/kaggle/input/sf-dst-predict-car-price/sample_submission.csv')
# sample_df.info()

In [8]:
# Some EDA
train_df = train_df_ini[(train_df_ini.final_price <= 100_000) & (train_df_ini.final_price >= 1)]

In [9]:
training_set, validation_set = train_test_split(
    train_df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

In [11]:
task = Task('reg', loss='mae', metric='mae')
roles = {
    'target': TARGET_NAME,
    'drop': ['row_ID']
}

In [12]:
automl = TabularAutoML(
    task=task,
    timeout=TIMEOUT,
    cpu_limit=N_THREADS,
    reader_params={'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [13]:
oof_pred = automl.fit_predict(
    training_set,
    roles=roles,
    verbose=0
)

In [14]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) 


In [15]:
val_pred = automl.predict(validation_set)
print(f'Prediction for validation_set:\n{val_pred}\nShape: {val_pred.shape}')

Prediction for validation_set:
array([[ 6164.416  ],
       [11619.204  ],
       [ 4329.804  ],
       ...,
       [12979.962  ],
       [  476.79437],
       [12725.819  ]], dtype=float32)
Shape: (6998, 1)


In [16]:
print(f'TRAIN out-of-fold score: {mean_absolute_error(training_set[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {mean_absolute_error(validation_set[TARGET_NAME].values, val_pred.data[:, 0])}')

TRAIN out-of-fold score: 1564.3136423244518
HOLDOUT score: 1452.4207840770457


In [17]:
test_pred = automl.predict(test_df)
print(f'Prediction result:\n{test_pred}\nShape: {test_pred.shape}')

Prediction result:
array([[ 2796.346 ],
       [ 5626.5674],
       [ 4383.2827],
       ...,
       [18132.66  ],
       [ 4954.5786],
       [ 6337.3354]], dtype=float32)
Shape: (10697, 1)


In [18]:
sample_df[TARGET_NAME] = test_pred.data[:, 0]
sample_df.to_csv('sample_submission.csv', index=False)